In [ ]:
# Step 1: Install required packages
!pip install gradio pandas scikit-learn matplotlib

In [ ]:
# Step 2: Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import gradio as gr
import matplotlib.pyplot as plt

# Step 3: Load your CSV dataset
from google.colab import files
uploaded = files.upload()  # Upload your CSV

Saving comment_dataset_1000_varied.csv to comment_dataset_1000_varied.csv


In [ ]:
duplicates = df[df.duplicated(subset='comment')]
print("Duplicate comments:")
print(duplicates)


Duplicate comments:
                                               comment                   label
38                         Stop making such animation!              Hate/Abuse
63                    Click here to get editing now!!!         Spam/Irrelevant
71   The music was okay, but could be improved in t...  Constructive Criticism
79   The animation was okay, but could be improved ...  Constructive Criticism
82                    Click here to get concept now!!!         Spam/Irrelevant
..                                                 ...                     ...
995    If you continue with design, I will report you.                  Threat
996             This graphics touches my heart deeply.               Emotional
997              Don't miss out, video available here!         Spam/Irrelevant
998                          Follow me for free theme!         Spam/Irrelevant
999                   Brilliant work on the storyline!                  Praise

[630 rows x 2 columns]


In [ ]:
df_no_duplicates = df.drop_duplicates(subset='comment', keep='first')
print("Dataset shape after removing duplicates:", df_no_duplicates.shape)

Dataset shape after removing duplicates: (370, 2)


In [ ]:
df = pd.read_csv(list(uploaded.keys())[0])
print("Dataset sample:")
print(df.head())

Dataset sample:
                                             comment                   label
0                  Could you do a guide on graphics?     Question/Suggestion
1      I liked the idea, but the storyline felt off.  Constructive Criticism
2     You can achieve anything, love your animation!                 Support
3                  Your editing is complete garbage!              Hate/Abuse
4  The voiceover has potential, but the execution...  Constructive Criticism


In [ ]:

# Step 4: Prepare features and labels
X = df['comment']
y = df['label']

# Optional: remove duplicates
df = df.drop_duplicates(subset='comment', keep='first')

# Step 5: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 6: Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Step 7: Train a Logistic Regression classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# Step 8: Evaluate the model
y_pred = clf.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Step 9: Prediction function for Gradio
reply_templates = {
    "Support": "Thank you very much for your support! 🙏",
    "Praise":" Thank you so much for your appreciation! Continue to support us! 💖",
    "Constructive Criticism": "I appreciate your input. That will be better! ✅",

    "Hate/Abuse":"Abuse is not acceptable to us. Please show some decency.",
    "Threat":" There is a serious threat. This has been documented.",
    "Emotional ":" We’re glad this resonated with you! ❤️" ,
    "Question/Suggestion" : "Thanks for the suggestion!",
     "Spam/Irrelevant":"This seems off-topic or spam.We'll think about it."
}

def classify_comment(comment):
    vec = vectorizer.transform([comment])
    pred = clf.predict(vec)[0]
    reply = reply_templates.get(pred, "")
    return pred, reply


Accuracy: 1.0
                        precision    recall  f1-score   support

Constructive Criticism       1.00      1.00      1.00         9
             Emotional       1.00      1.00      1.00         8
            Hate/Abuse       1.00      1.00      1.00        10
                Praise       1.00      1.00      1.00        10
   Question/Suggestion       1.00      1.00      1.00         9
       Spam/Irrelevant       1.00      1.00      1.00         9
               Support       1.00      1.00      1.00        10
                Threat       1.00      1.00      1.00         9

              accuracy                           1.00        74
             macro avg       1.00      1.00      1.00        74
          weighted avg       1.00      1.00      1.00        74



In [ ]:

import gradio as gr

# Updated classify_comment function with colored output
def classify_comment(comment):
    vec = vectorizer.transform([comment])
    pred = clf.predict(vec)[0]
    reply = reply_templates.get(pred, "")

    # Map category to colors
    colors = {
        "Praise": "green",
        "Support": "blue",
        "Constructive Criticism": "orange",
        "Hate/Abuse": "red",
        "Threat": "darkred",
        "Emotional": "pink",
        "Spam/Irrelevant": "gray",
        "Question/Suggestion": "purple"
    }
    color = colors.get(pred, "black")

    # Return HTML colored category
    colored_pred = f"<span style='color:{color}; font-weight:bold;'>{pred}</span>"
    return colored_pred, reply

# Gradio Interface with mascot and colors
iface = gr.Interface(
    fn=classify_comment,
    inputs=gr.Textbox(
        lines=5,
        placeholder="Type your comment here...",
        label="Your Comment"
    ),
    outputs=[
        gr.HTML(label="Predicted Category"),  # Use HTML to show colored text
        gr.Textbox(label="Suggested Reply")
    ],
    title="🐼 Comment Categorization & Reply Assistant 🐼",
    description="""
    <div style='display:flex; align-items:center;'>
        Enter a comment and get its predicted category along with a suggested reply.
    </div>
    """,
    theme="default"
)

# Launch the interface
iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c639d0f31577b6d817.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
